## Inference with koBEST Benchmark Dataset

In [ ]:
from huggingface_hub import login
login(token='************')

In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto", max_length=256
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
pipeline("콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 1998년 4월 1일에 출시한 탄산음료다. 질문: 콜라독립 815는 코카콜라에서 만들었나요? 답변:")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 1998년 4월 1일에 출시한 탄산음료다. 질문: 콜라독립 815는 코카콜라에서 만들었나요? 답변: 콜라독립 815은 범양식품이 만든 제품입니다. 콜라독립 815은 1998년 4월 1일에 출시한 탄산음료다. 콜라독립 815의 맛은 코카콜라와 비슷하다. 콜라독립 815은 코카콜라보다 가격이 저렴하다. 콜라독립 815은 코카콜라와 같은 음료를 만들기 위해 코카콜라의 레시피를 사용했다. 콜라독립 815은 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 만든 제품이다. 콜라독립 815은 코카콜라와 비슷한 음료를 만들기 위해 코카콜라의 레시피를 사용했다. 콜라독립 815은 코카콜라보다 가격이'}]

## Install lm-evaluation-harness

In [4]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git to /tmp/pip-req-build-7drezq37
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git /tmp/pip-req-build-7drezq37
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit a4987bba6e9e9b3f22bd3a6c1ecf0abd04fd5622
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Evaluate KoBEST boolq 

In [5]:
!ls ./lm-evaluation-harness/lm_eval/tasks/kobest

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


kobest_boolq.yaml  kobest_hellaswag.yaml  kobest_wic.yaml  README.md
kobest_copa.yaml   kobest_sentineg.yaml   __pycache__	   utils.py


In [6]:
!cat ./lm-evaluation-harness/lm_eval/tasks/kobest/kobest_boolq.yaml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tag:
  - kobest
task: kobest_boolq
dataset_path: skt/kobest_v1
dataset_name: boolq
output_type: multiple_choice
training_split: train
validation_split: validation
test_split: test
doc_to_text: "{{paragraph}} 질문: {{question}} 답변: "
doc_to_target: "{{label}}"
doc_to_choice: ["아니오", "예"]
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: True
  - metric: f1
    aggregation: !function utils.macro_f1_score
    average: macro
    hf_evaluate: true
    higher_is_better: True
metadata:
  version: 1.0


In [7]:
!lm_eval --model hf \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B,dtype=auto \
    --tasks kobest_boolq \
    --num_fewshot 0 \
    --seed 42 \
    --device cuda:0 \
    --batch_size auto\
    --output_path ./eval_llama31-8B/kobest/boolq/0-shot \
    --log_samples \
    --show_config

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024-08-22:10:15:38,195 INFO     [__main__.py:272] Verbosity set to INFO
2024-08-22:10:15:38,198 INFO     [__init__.py:491] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-08-22:10:15:42,993 INFO     [__main__.py:376] Selected Tasks: ['kobest_boolq']
2024-08-22:10:15:42,994 INFO     [evaluator.py:158] Setting random seed to 42 | Setting numpy seed to 42 | Setting torch manual seed to 42
2024-08-22:10:15:42,994 INFO     [evaluator.py:195] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Meta-Llama-3.1-8B', 'dtype': 'auto'}
2024-08-22:10:15:43,023 INFO     [huggingface.py:130] Using device 'cuda:0'
2024-08-22:10:15:43,328 INFO     [huggingface.py:366] Model para

In [8]:
!ls ./eval_llama31-8B/kobest/boolq/0-shot/meta-llama__Meta-Llama-3.1-8B

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


results_2024-08-22T10-16-39.908535.json
samples_kobest_boolq_2024-08-22T10-16-39.908535.jsonl


In [9]:
!head -n 1 ./eval_llama31-8B/kobest/boolq/0-shot/meta-llama__Meta-Llama-3.1-8B/samples_kobest_boolq_2024-08-22T10-16-39.908535.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"doc_id": 0, "doc": {"paragraph": "콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 1998년 4월 1일에 출시한 탄산음료다.", "question": "콜라독립 815는 코카콜라에서 만들었나요?", "label": 0}, "target": "0", "arguments": {"gen_args_0": {"arg_0": "콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 1998년 4월 1일에 출시한 탄산음료다. 질문: 콜라독립 815는 코카콜라에서 만들었나요? 답변: ", "arg_1": " 아니오"}, "gen_args_1": {"arg_0": "콜라독립 815(815 Cola)는 코카콜라를 대한민국에서 라이선스로 만들어 오던 범양식품이 1998년 4월 1일에 출시한 탄산음료다. 질문: 콜라독립 815는 코카콜라에서 만들었나요? 답변: ", "arg_1": " 예"}}, "resps": [[["-14.689537048339844", "False"]], [["-15.082908630371094", "False"]]], "filtered_resps": [["-14.689537048339844", "False"], ["-15.082908630371094", "False"]], "doc_hash": "821038daf427b97b5fb456244a774a9a8585275803d8dfe2747d18ae34574dce", "prompt_hash": "43134ceb9b23f5061fefdfced4926c5aca808acccd8447e684440e1fd4ec844f", "target_hash": "5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9", "acc": 1.0, "f1": [0, 0]}


## Evaluate KMMLU direct_food_processing

In [10]:
!ls ./lm-evaluation-harness/lm_eval/tasks/kmmlu/direct

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


_direct_kmmlu_yaml
kmmlu_direct_accounting.yaml
kmmlu_direct_agricultural_sciences.yaml
kmmlu_direct_aviation_engineering_and_maintenance.yaml
kmmlu_direct_biology.yaml
kmmlu_direct_chemical_engineering.yaml
kmmlu_direct_chemistry.yaml
kmmlu_direct_civil_engineering.yaml
kmmlu_direct_computer_science.yaml
kmmlu_direct_construction.yaml
kmmlu_direct_criminal_law.yaml
kmmlu_direct_ecology.yaml
kmmlu_direct_economics.yaml
kmmlu_direct_education.yaml
kmmlu_direct_electrical_engineering.yaml
kmmlu_direct_electronics_engineering.yaml
kmmlu_direct_energy_management.yaml
kmmlu_direct_environmental_science.yaml
kmmlu_direct_fashion.yaml
kmmlu_direct_food_processing.yaml
kmmlu_direct_gas_technology_and_engineering.yaml
kmmlu_direct_geomatics.yaml
kmmlu_direct_health.yaml
kmmlu_direct_industrial_engineer.yaml
kmmlu_direct_information_technology.yaml
kmmlu_direct_interior_architecture_and_design.yaml
kmmlu_direct_korean_history.yaml
kmmlu_direct_law.yaml
kmmlu_direct_machine_design_and_manufacturi

In [11]:
!cat ./lm-evaluation-harness/lm_eval/tasks/kmmlu/direct/_direct_kmmlu_yaml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tag:
    - kmmlu
    - kmmlu_direct
dataset_path: HAERAE-HUB/KMMLU
output_type: generate_until
test_split: test
fewshot_split: dev
doc_to_text: "{{question.strip()}}\nA. {{A}}\nB. {{B}}\nC. {{C}}\nD. {{D}}\n정답："
doc_to_target: "{{['A', 'B', 'C', 'D'][answer-1]}}"
metric_list:
  - metric: exact_match
    aggregation: mean
    higher_is_better: true
    ignore_case: true
    ignore_punctuation: true
    regexes_to_ignore:
          - " "
generation_kwargs:
  until:
    - "Q:"
    - "\n\n"
    - "</s>"
    - "."
  do_sample: false
  temperature: 0.0
metadata:
  version: 2.0


In [12]:
!lm_eval --model hf \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B,dtype=auto \
    --tasks kmmlu_direct_food_processing \
    --num_fewshot 0 \
    --seed 42 \
    --device cuda:0 \
    --batch_size auto\
    --output_path ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot \
    --log_samples \
    --show_config 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024-08-22:10:17:32,309 INFO     [__main__.py:272] Verbosity set to INFO
2024-08-22:10:17:32,311 INFO     [__init__.py:491] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-08-22:10:17:37,468 INFO     [__main__.py:376] Selected Tasks: ['kmmlu_direct_food_processing']
2024-08-22:10:17:37,470 INFO     [evaluator.py:158] Setting random seed to 42 | Setting numpy seed to 42 | Setting torch manual seed to 42
2024-08-22:10:17:37,470 INFO     [evaluator.py:195] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Meta-Llama-3.1-8B', 'dtype': 'auto'}
2024-08-22:10:17:37,516 INFO     [huggingface.py:130] Using device 'cuda:0'
2024-08-22:10:17:37,803 INFO     [huggingface.py

In [13]:
!ls ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot/meta-llama__Meta-Llama-3.1-8B/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


results_2024-08-22T10-37-31.530806.json
samples_kmmlu_direct_food_processing_2024-08-22T10-37-31.530806.jsonl


In [14]:
!head -n 5 ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot/meta-llama__Meta-Llama-3.1-8B/samples_kmmlu_direct_food_processing_2024-08-22T10-37-31.530806.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"doc_id": 0, "doc": {"question": "산성암, 중성암, 염기성암의 분류기분이 되는 것은?", "answer": 2, "A": "SiO2", "B": "Al2O3", "C": "P2O5", "D": "Fe2O3", "Category": "Food Processing", "Human Accuracy": 0.0435}, "target": "B", "arguments": {"gen_args_0": {"arg_0": "산성암, 중성암, 염기성암의 분류기분이 되는 것은?\nA. SiO2\nB. Al2O3\nC. P2O5\nD. Fe2O3\n정답：", "arg_1": {"until": ["Q:", "\n\n", "</s>", "."], "do_sample": false, "temperature": 0.0}}}, "resps": [["A\n해설：SiO2의 함량이 52% 이상인 것은 중성암, 52% 미만인 것은 염기성암, 52% 이상인 것은 중성암, 52% 미만인 것은 염기성암이다"]], "filtered_resps": ["A\n해설：SiO2의 함량이 52% 이상인 것은 중성암, 52% 미만인 것은 염기성암, 52% 이상인 것은 중성암, 52% 미만인 것은 염기성암이다"], "doc_hash": "139b202ef728343bfaa798fae16ca033d6ff3cced5c0243ebd7fae6d4e99e0d5", "prompt_hash": "70463d87a5f8877b36f1402a45a5d56a498cc7eaadb67fbb6e9faadddde1e5ab", "target_hash": "df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c", "exact_match": 0.0}
{"doc_id": 1, "doc": {"question": "어류의 생식선 자극 호르몬에 대한 설명 중 바르게 된 것은?", "answer": 3, "A": "뇌하수체의 전엽단부의 대부분을 차지하며, 에리드로신에

In [15]:
!mkdir template

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
YAML_kmmlu_direct_food_processing = '''
task: kmmlu_direct_food_processing_filtered
dataset_path: HAERAE-HUB/KMMLU
dataset_name: Food-Processing
output_type: generate_until
test_split: test
fewshot_split: dev
doc_to_text: "{{question.strip()}}\nA. {{A}}\nB. {{B}}\nC. {{C}}\nD. {{D}}\n정답："
doc_to_target: "{{['A', 'B', 'C', 'D'][answer-1]}}"
metric_list:
  - metric: exact_match
    aggregation: mean
    higher_is_better: true
    ignore_case: true
    ignore_punctuation: true
    regexes_to_ignore:
          - " "
generation_kwargs:
  until:
    - "Q:"
    - "\n\n"
    - "</s>"
    - "."
  do_sample: false
  temperature: 0.0
filter_list:
  - name: "get-answer"
    filter:
      - function: "regex"
        regex_pattern: "[A-D]"
      - function: "take_first"
metadata:
  version: 2.0
'''
with open('./template/kmmlu_direct_food_processing_filtered.yaml', 'w') as f:
    f.write(YAML_kmmlu_direct_food_processing)

In [17]:
!lm_eval --model hf \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B,dtype=auto \
    --include_path ./template \
    --tasks kmmlu_direct_food_processing_filtered \
    --num_fewshot 0 \
    --seed 42 \
    --device cuda:0 \
    --batch_size auto\
    --output_path ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot \
    --log_samples \
    --show_config 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024-08-22:10:54:15,772 INFO     [__main__.py:272] Verbosity set to INFO
2024-08-22:10:54:15,772 INFO     [__main__.py:303] Including path: ./template
2024-08-22:10:54:15,774 INFO     [__init__.py:491] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-08-22:10:54:20,777 INFO     [__main__.py:376] Selected Tasks: ['kmmlu_direct_food_processing_filtered']
2024-08-22:10:54:20,778 INFO     [evaluator.py:158] Setting random seed to 42 | Setting numpy seed to 42 | Setting torch manual seed to 42
2024-08-22:10:54:20,778 INFO     [evaluator.py:195] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Meta-Llama-3.1-8B', 'dtype': 'auto'}
2024-08-22:10:54:20,813 INFO     [hug

In [18]:
!ls ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot/meta-llama__Meta-Llama-3.1-8B/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


results_2024-08-22T10-37-31.530806.json
results_2024-08-22T10-56-36.453767.json
samples_kmmlu_direct_food_processing_2024-08-22T10-37-31.530806.jsonl
samples_kmmlu_direct_food_processing_filtered_2024-08-22T10-56-36.453767.jsonl


In [19]:
!head -n 5 ./eval_llama31-8B/kmmlu/direct_food_processing/0-shot/meta-llama__Meta-Llama-3.1-8B/samples_kmmlu_direct_food_processing_filtered_2024-08-22T10-56-36.453767.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"doc_id": 0, "doc": {"question": "산성암, 중성암, 염기성암의 분류기분이 되는 것은?", "answer": 2, "A": "SiO2", "B": "Al2O3", "C": "P2O5", "D": "Fe2O3", "Category": "Food Processing", "Human Accuracy": 0.0435}, "target": "B", "arguments": {"gen_args_0": {"arg_0": "산성암, 중성암, 염기성암의 분류기분이 되는 것은? A. SiO2 B. Al2O3 C. P2O5 D. Fe2O3 정답：", "arg_1": {"until": ["Q:", "\n", "</s>", "."], "do_sample": false, "temperature": 0.0}}}, "resps": [["A"]], "filtered_resps": ["A"], "doc_hash": "139b202ef728343bfaa798fae16ca033d6ff3cced5c0243ebd7fae6d4e99e0d5", "prompt_hash": "f38ca08879a428518f5acdabe20340fdb6eaa106e4edd2b3c6da4b19460dd80d", "target_hash": "df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c", "exact_match": 0.0}
{"doc_id": 1, "doc": {"question": "어류의 생식선 자극 호르몬에 대한 설명 중 바르게 된 것은?", "answer": 3, "A": "뇌하수체의 전엽단부의 대부분을 차지하며, 에리드로신에 잘 염색되는 세포에서 생산, 분비된다.", "B": "어류의 간신선을 자극하여 코티졸이나 코티존의 생산, 분비를 촉진한다.", "C": "뇌하수체 전엽주부에서 생산, 분비되고 당단백질을 함유한 과립을 세포질 가운데 많이 가진다.", "D": "뇌하수체 중엽에서 생산, 분비되는 호르몬이다.", "Cat